# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd  
import sqlite3

conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [2]:
# Your code here
q1 = """ 
SELECT lastName, firstName
FROM employees
JOIN offices
ON employees.officeCode = offices.officeCode;
"""
pd.read_sql(q1, conn)

,lastName,firstName
0,Murphy,Diane
1,Patterson,Mary
2,Firrelli,Jeff
3,Patterson,William
4,Bondur,Gerard
5,Bow,Anthony
6,Jennings,Leslie
7,Thompson,Leslie
8,Firrelli,Julie
9,Patterson,Steve


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [3]:
# Your code here
q2 = """ 
SELECT officeCode, city, COUNT(employeeNumber) AS numberofemployees
FROM offices
JOIN employees
USING (officeCode)
GROUP BY officeCode, city;
"""
pd.read_sql(q2, conn)

,officeCode,city,numberofemployees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


## Write 3 questions of your own and answer them

1.Which customers have the highest order quantities? Limit to 10. Select columns customerName, orderNumber, quantityOrdered.

In [24]:

q3 = """
SELECT customerName, orderNumber, SUM(quantityOrdered)
FROM customers
JOIN orders USING (customerNumber)
JOIN orderdetails USING (orderNumber)
GROUP BY quantityOrdered
ORDER BY customerName, orderNumber DESC
LIMIT 10;
"""
pd.read_sql(q3, conn)

,customerName,orderNumber,SUM(quantityOrdered)
0,Atelier graphique,10345,3870
1,Atelier graphique,10298,3072
2,Atelier graphique,10298,3471
3,Atelier graphique,10123,2522
4,Atelier graphique,10123,4114
5,Atelier graphique,10123,4784
6,Atelier graphique,10123,3650
7,"Australian Collectors, Co.",10347,2808
8,"Australian Collectors, Co.",10342,2490
9,"Australian Collectors, Co.",10342,3520


2. Which customers have the highest order quantities? Limit to 10. Select columns customerName, orderNumber, quantityOrdered. Get the 10 highest customers with highest quantityOrdered.

In [29]:

q4 = """
SELECT customerName, orderNumber, SUM(quantityOrdered)
FROM customers
JOIN orders USING (customerNumber)
JOIN orderdetails USING (orderNumber)
GROUP BY  customerName 
ORDER BY   quantityOrdered ASC
LIMIT 10;
"""
pd.read_sql(q4, conn)


,customerName,orderNumber,SUM(quantityOrdered)
0,"AV Stores, Co.",10110,1778
1,Amica Models & Co.,10280,843
2,"Australian Collectables, Ltd",10193,705
3,"Clover Collections, Co.",10220,490
4,Gifts4AllAges.com,10264,1036
5,Heintze Collectables,10161,882
6,Herkku Gifts,10181,973
7,L'ordine Souveniers,10176,1280
8,Land of Toys Inc.,10107,1631
9,Lyon Souveniers,10134,684


3. Find the customers who brought the highest revenue to the company so far. limit to 10.

In [33]:
q5 = """
SELECT customerName, SUM(amount)
FROM customers
JOIN payments USING (customerNumber)
GROUP BY customerName
ORDER BY SUM(amount) DESC
LIMIT 10;
"""

pd.read_sql(q5, conn)

,customerName,SUM(amount)
0,Euro+ Shopping Channel,715738.98
1,Mini Gifts Distributors Ltd.,584188.24
2,"Australian Collectors, Co.",180585.07
3,Muscle Machine Inc,177913.95
4,"Dragon Souveniers, Ltd.",156251.03
5,"Down Under Souveniers, Inc",154622.08
6,"AV Stores, Co.",148410.09
7,"Anna's Decorations, Ltd",137034.22
8,Corporate Gift Ideas Co.,132340.78
9,"Saveley & Henriot, Co.",130305.35


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [8]:
# Your code here
q7 = """ 
SELECT firstName, lastName, productName
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders USING (customerNumber)
JOIN orderdetails USING (orderNumber)
JOIN products USING (productCode);
"""
pd.read_sql(q7, conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [9]:
# Your code here
q8 = """
SELECT firstName, lastName, COUNT(quantityOrdered) AS numberofproducts
FROM employees 
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders USING (customerNumber)
JOIN orderdetails USING (orderNumber)
GROUP BY employeeNumber, firstName, lastName
ORDER BY lastName;
"""
pd.read_sql(q8, conn)

,firstName,lastName,numberofproducts
0,Loui,Bondur,177
1,Larry,Bott,236
2,Pamela,Castillo,272
3,Julie,Firrelli,124
4,Andy,Fixter,185
5,Martin,Gerard,114
6,Gerard,Hernandez,396
7,Leslie,Jennings,331
8,Barry,Jones,220
9,Peter,Marsh,185


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [10]:
# Your code here
q9 = """
SELECT firstName, lastName, COUNT(DISTINCT productCode) AS numberofdiffproducts
FROM employees 
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders USING (customerNumber)
JOIN orderdetails USING (orderNumber)
JOIN products USING (productCode)
GROUP BY employeeNumber, firstName, lastName
HAVING COUNT (DISTINCT productCode) > 200;
"""
pd.read_sql(q9, conn)

,firstName,lastName,numberofdiffproducts


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!